In [1]:
import pickle
import pandas as pd

In [42]:
import pickle


archivo_pickle = 'Entrenamiento_cronicas_N.pkl'


with open(archivo_pickle, 'rb') as file:
    df = pickle.load(file)


In [43]:
pd.set_option('display.max_columns', None)
df.shape

(996234, 45)

In [44]:
# Eliminar columnas donde todas las entradas son cero
data_non_zero = df.loc[:, (df != 0).any(axis=0)]

# Mostrar las columnas eliminadas y el nuevo dataframe
columns_removed = df.columns.difference(data_non_zero.columns)


In [45]:
import pandas as pd

# Asumiendo que 'data_non_zero' es tu DataFrame actual
# Lista de todas las posibles columnas de enfermedades
columns_enfermedades = [
    'SIDA_VIH', 'TUBERCULOSIS', 'CANCER_OTROSCANCER', 'CANCER_ORGANOSDIGESTIVOS',
    'CANCER_ORGANOSRESPIRATORIOS', 'CANCER_MELANOMAYDELAPIEL', 'CANCER_MAMA',
    'CANCER_OTROSGENITALESFEMENINOS', 'CANCER_CERVIXINVASIVO', 'CANCER_GENITALESMASCULINOS',
    'CANCER_TEJIDOLINFATICOYAFINES', 'CANCER_CERVIXINSITU', 'AUTOINMUNE', 'DIABETES',
    'SINDROMESCONVULSIVOS(EPILEPSIA)', 'ENFERMEDADCARDIOVASCULAR_OTRA', 'ENFERMEDADCARDIOVASCULAR_HIPERTENSION',
    'ENFERMEDADPULMONARDELARGADURACION', 'ASMA', 'ARTRITISPIOGENASYREACTIVAS',
    'ARTRITIS', 'ARTROSIS', 'RENAL_OTRA', 'RENAL_RENALDELARGADURACION', 'TRANSPLANTE',
    'RENAL_INSUFICIENCIARENALOTRA', 'RENAL_INSUFICIENCIARENALCRONICA', 'ANOMALIASGENETICASYCONGENITAS',
    'CANCER_TERAPIACANCER'
]


# Verificar cuáles columnas de enfermedades están presentes en el DataFrame
columns_present = [col for col in columns_enfermedades if col in data_non_zero.columns]

# Crear la columna 'Enfermedad' que concatena los nombres de las enfermedades presentes, separadas por comas
data_non_zero['Enfermedad'] = (data_non_zero[columns_present] > 0).apply(lambda x: ', '.join(x.index[x]), axis=1)

# Mostrar las primeras filas para verificar la nueva columna 'Enfermedad'
data_non_zero[['Enfermedad']].head()




/var/folders/tv/6r5469xx31vbksvh_943nnr1bcv0mh/T/ipykernel_58807/1138452748.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_non_zero['Enfermedad'] = (data_non_zero[columns_present] > 0).apply(lambda x: ', '.join(x.index[x]), axis=1)


,Enfermedad
rownames,
1184873,
1587467,ENFERMEDADCARDIOVASCULAR_OTRA
753062,
1802563,
1190692,


In [46]:
data_non_zero.drop(columns=columns_enfermedades, inplace=True)

# Reemplazar valores vacíos o NaN en la columna 'Enfermedad' con 'Sin Enfermedad'
data_non_zero['Enfermedad'] = data_non_zero['Enfermedad'].apply(lambda x: 'Sin Enfermedad' if not x else x)


/var/folders/tv/6r5469xx31vbksvh_943nnr1bcv0mh/T/ipykernel_58807/3052532982.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_non_zero.drop(columns=columns_enfermedades, inplace=True)
/var/folders/tv/6r5469xx31vbksvh_943nnr1bcv0mh/T/ipykernel_58807/3052532982.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_non_zero['Enfermedad'] = data_non_zero['Enfermedad'].apply(lambda x: 'Sin Enfermedad' if not x else x)


In [47]:
df = data_non_zero
df.drop(columns='NUM_IDE', inplace=True)
df.drop(columns='FECHA_NACI', inplace=True)
df.drop(columns='id', inplace=True)
df.head()

,EPS,SEXO,DPTO,MUNI,DIAS_COMP,V8,Valor_Neto,DIAS_ESTAN_Neto,Edad,Grupo_Edad,Zona,Enfermedad
rownames,,,,,,,,,,,,
1184873,EPS002,F,50,1,360,B,39054.0,0.0,12.0,3,N,Sin Enfermedad
1587467,EPS013,M,76,1,336,C,312012.0,277.0,23.0,5,C,ENFERMEDADCARDIOVASCULAR_OTRA
753062,EPS017,F,11,1,333,B,14313.0,0.0,15.0,4,C,Sin Enfermedad
1802563,EPS002,M,76,1,84,B,0.0,0.0,13.0,3,C,Sin Enfermedad
1190692,EPS037,F,15,776,180,B,0.0,0.0,21.0,5,N,Sin Enfermedad


In [ ]:
Categoricas: EPS SEXO DPTO MUNI V8 Grupo_Edad Zona Enfermedad
Numericas: Valor_Neto DIAS_ESTAN_Neto	Edad

In [55]:
df.Grupo_Edad.value_counts()

Grupo_Edad
5     447291
3     156481
4      67207
6      65305
2      54074
7      53909
8      41770
9      31761
12     27224
10     22602
11     17587
1      11023
Name: count, dtype: int64

In [35]:
import numpy as np

# Laplace Mechanism
def laplace_mechanism(value, epsilon):
    scale = 1/epsilon
    return value + np.random.laplace(0, scale)



def oue(data, column_name, epsilon):
    categories = data[column_name].unique()
    def encode(category):
        p = 1 / (np.exp(epsilon) + 1)
        q = 1 - p
        encoded = [1 if np.random.rand() < p else 0 if cat == category else 1 if np.random.rand() < q else 0 for cat in categories]
        max_index = np.argmax(encoded)
        return categories[max_index]
    return data[column_name].apply(encode)





          Valor_Neto  Valor_Neto_Laplace SEXO Sexo_OUE
rownames                                              
1184873      39054.0        39054.150381    F        M
1587467     312012.0       312012.680544    M        F
753062       14313.0        14312.026919    F        M
1802563          0.0           -0.353630    M        F
1190692          0.0            1.152578    F        F


In [ ]:
import pandas as pd
import numpy as np
import pickle

# Definir las funciones de mecanismo de privacidad
def laplace_mechanism(value, epsilon):
    """Añade ruido laplaciano a un valor numérico."""
    scale = 1 / epsilon
    return value + np.random.laplace(0, scale)

def oue(data, column_name, epsilon):
    """Aplica Optimized Unary Encoding a una columna categórica."""
    categories = data[column_name].unique()
    def encode(category):
        p = 1 / (np.exp(epsilon) + 1)
        q = 1 - p
        encoded = [1 if np.random.rand() < p else 0 if cat == category else 1 if np.random.rand() < q else 0 for cat in categories]
        max_index = np.argmax(encoded)
        return categories[max_index]
    return data[column_name].apply(encode)

# Listas de columnas categóricas y numéricas
categorical_columns = ['EPS', 'SEXO', 'DPTO', 'MUNI', 'V8', 'Grupo_Edad', 'Zona', 'Enfermedad']
numeric_columns = ['Valor_Neto', 'DIAS_ESTAN_Neto', 'Edad']

# Valores de epsilon a usar
epsilons = [0.01, 0.1, 0.5, 1.0]

# Aplicar mecanismos de privacidad y guardar DataFrames
for epsilon in epsilons:
    df_epsilon = df.copy()
    
    # Aplicar Laplace a columnas numéricas
    for col in numeric_columns:
        df_epsilon[col] = df_epsilon[col].apply(lambda x: laplace_mechanism(x, epsilon))
    
    # Aplicar OUE a columnas categóricas
    for col in categorical_columns:
        df_epsilon[col] = oue(df_epsilon, col, epsilon)
    
    # Guardar el DataFrame resultante como un archivo pickle
    pickle_filename = f'base_epsilon_{epsilon}.pkl'
    with open(pickle_filename, 'wb') as f:
        pickle.dump(df_epsilon, f)

    print(f'DataFrame para epsilon={epsilon} guardado como {pickle_filename}.')
